In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.5"
export JULIA_PKG_PRECOMPILE_AUTO=0
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  echo "Installing PyCall.jl..."
  julia -e 'using Pkg; Pkg.add("PyCall"); Pkg.build("PyCall")'
  julia -e 'println("Success")'

fi

Installing Julia 1.8.5 on the current Colab Runtime...
2023-11-09 11:20:13 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.8/julia-1.8.5-linux-x86_64.tar.gz [130873886/130873886] -> "/tmp/julia.tar.gz" [1]
Installing PyCall.jl...
  Installing known registries into `~/.julia`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Conda ─────────── v1.9.1
   Installed VersionParsing ── v1.3.0
   Installed Preferences ───── v1.4.1
   Installed MacroTools ────── v0.5.11
   Installed Parsers ───────── v2.7.2
   Installed JSON ──────────── v0.21.4
   Installed PrecompileTools ─ v1.2.0
   Installed PyCall ────────── v1.96.2
    Updating `~/.julia/environments/v1.8/Project.toml`
  [438e738f] + PyCall v1.96.2
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8f4d0f93] + Conda v1.9.1
  [682c06a0] + JSON v0.21.4
  [1914dd2f] + MacroTools v0.5.11
  [69de0a69] + Parsers v2.7.2
  [aea7be01] + PrecompileTools v1.2.0
  [21216c6a] + Pre

In [2]:
%pip install -Uq pysr pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 14.2 MB/s eta 0:00:00


In [3]:
from julia import Julia

julia = Julia(compiled_modules=False, threads="auto")
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()

In [4]:
import pysr

# We don't precompile in colab because compiled modules are incompatible static Python libraries:
pysr.install(precompile=False)

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      Ubuntu 22.04.2 LTS
  uname: Linux 5.15.120+ #1 SMP Wed Aug 30 11:19:59 UTC 2023 x86_64 x86_64
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz: 
              speed         user         nice          sys         idle          irq
       #1  2199 MHz        572 s          0 s        168 s       2040 s          0 s
       #2  2199 MHz        692 s          0 s        177 s       1906 s          0 s
  Memory: 12.6783447265625 GB (11580.0859375 MB free)
  Uptime: 287.08 sec
  Load Avg:  0.53  0.51  0.25
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 1 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_PROJECT = @pysr-0.16.3
  JULIA_PKG_PRECOMPILE_AUTO = 0
  TCLLIBPATH = /usr/share/tcltk/tcllib1.20
  HOME = /root
  PYTHONPATH = /env/python
  LIBRARY_PATH = /usr/local/cuda/lib64/stubs
  PATH = /opt/bin:/usr/local/nvidia/b

[ Info: Julia version info
[ Info: Julia executable: /usr/local/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /usr/bin/python3
│     libpython: /usr/lib/x86_64-linux-gnu/libpython3.10.so.1.0
│ Python:
│     python: /usr/bin/python3
└     libpython: 
    Updating registry at `~/.julia/registries/General.toml`
     Cloning git-repo `https://github.com/MilesCranmer/SymbolicRegression.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Tricks ─────────────────────────── v0.1.8
   Installed IrrationalConstants ────────────── v0.2.2
   Installed ScientificTypesBase ────────────── v3.0.0
   Installed Adapt ──────────────────────────── v3.7.1
   Installed DiffRules ──────────────────────── v1.15.1
   Installed SIMDTypes ──────────────────────── v0.1.0
   Installed BitTwiddlingConvenienceFunctions ─ v0.1.5
   Installed DynamicExpressions 

In [10]:
import sympy
import numpy as np
import math
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split

In [18]:
x = np.arange(1,61)
y = [(i**2)/(math.sin(i)) for i in range(1,61)]

In [19]:
x

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [20]:
y[:10] # sample data

[1.1883951057781212,
 4.399000681178466,
 63.77550656163468,
 -21.141579340974438,
 -26.070880319285145,
 -128.8403837011586,
 74.58295206562279,
 64.68839797760621,
 196.54541812771114,
 -183.81639608896657]

In [21]:
x = np.expand_dims(x,axis=1)
y = np.expand_dims(y,axis=1)

In [24]:
model = PySRRegressor(
    binary_operators=['+', '-', '*', '/', '^'],
    unary_operators=['square', 'cube', 'exp', 'log', 'log10', 'log2', 'sqrt', 'sin', 'cos', 'tan'],
    niterations=600,
    verbosity=0
)

In [25]:
model.fit(x,y)

/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/julia_helpers.py:231: UserWarning: Julia has already started. The new Julia options {'threads': 2} will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:109: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(


PySRRegressor.equations_ = [
	   pick      score                equation          loss  complexity
	0         0.000000                919.6537  2.657211e+08           1
	1         0.009989              square(x0)  2.630800e+08           2
	2         0.000190        (x0 ^ 2.0321984)  2.630299e+08           3
	3         0.983833    (784.6971 / sin(x0))  9.834036e+07           4
	4  >>>>  36.579118  (square(x0) / sin(x0))  1.278271e-08           5
]

In [27]:
model.sympy()

x0**2/sin(x0)

In [28]:
for x0 in range(1,61):
  res = (np.square(x0) / np.sin(x0))
  print(res)

1.1883951057781212
4.399000681178466
63.77550656163468
-21.141579340974438
-26.070880319285145
-128.8403837011586
74.58295206562279
64.68839797760621
196.54541812771114
-183.81639608896657
-121.0011850189703
-268.36986208067117
402.2209863876796
197.85841370268653
346.00062634669206
-889.187394453742
-300.60407109550243
-431.4320934114428
2408.6383834595395
438.1423745632013
527.0985811697185
-54681.175871317406
-625.1326455731445
-636.0576004970698
-4722.264719116218
886.4894219909795
762.2525602627533
2893.9950116866034
-1267.2650086235103
-910.9020177631602
-2378.491239922935
1857.0011840815882
1089.0959928039815
2184.913682353533
-2860.9284944705396
-1306.742925099414
-2127.302065005298
4872.311384318394
1578.1357969186947
2147.324842538927
-10597.476468319872
-1924.668333234187
-2222.9576172952034
109366.63602807444
2379.823260349892
2346.4485935274593
17876.05549593814
-2999.0055589513977
-2517.4241906989605
-9528.352144680262
3880.7621239810446
2740.649077538715
7094.77536021626